In [3]:
import sys
sys.path.append('../')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from exploration.plotting import CandlePlot

In [5]:
def data_load(path):
    data = pd.read_pickle(path)
    data['spread'] = data['ask_o'] - data['bid_o']
    data.drop(columns=['bid_o', 'ask_o', 'bid_h', 'ask_h', 'bid_l', 'ask_l', 'bid_c', 'ask_c'], inplace=True)

    return data

In [6]:
BUY = 1
NONE = 0
SELL = -1

In [7]:
df = data_load('../data/USD_JPY_H1.pkl')

In [8]:
df.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'spread'], dtype='object')

In [9]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,spread
0,2019-01-01 22:00:00+00:00,482,109.590,109.698,109.590,109.684,0.100
1,2019-01-01 23:00:00+00:00,621,109.681,109.726,109.648,109.672,0.026
2,2019-01-02 00:00:00+00:00,796,109.674,109.682,109.533,109.629,0.016
3,2019-01-02 01:00:00+00:00,717,109.626,109.676,109.584,109.598,0.016
4,2019-01-02 02:00:00+00:00,1181,109.600,109.613,109.338,109.412,0.015


In [10]:
# Let's see the high and low value for spread
df['spread'].describe()

count    36845.000000
mean         0.019398
std          0.015903
min          0.010000
25%          0.013000
50%          0.015000
75%          0.018000
max          0.149000
Name: spread, dtype: float64

In [58]:
# Function for identifying the candle as hammar
def check_for_hammer(row, df):
    """
    row: row is the current candle
    df: Data frame contain of last 200 candles, I have to check for the last one to check if it is hammer or not.
    If last candle is hammer, then I check for other conditions.
    The last 10 days MA should be above and 200 MA should be below.
    """

    close_price = row['mid_c']
    open_price = row['mid_o']
    high_price = row['mid_h']
    low_price = row['mid_l']

    min_body_ratio = 0.15
    shadow_to_body_ratio = 2
    upper_shadow_ratio = 0.1


    # Check if the candle is green (close > open)
    # if close_price <= open_price:
    #     return NONE

    # Calculate the real body and shadows
    real_body = abs(close_price - open_price)
    lower_shadow = open_price - low_price
    upper_shadow = high_price - close_price
    total_range = high_price - low_price
    
    if total_range == 0 or real_body == 0:  # Avoid division by zero
        return NONE
    if real_body / total_range > min_body_ratio:
        return NONE

    # Check if the lower shadow is at least shadow_to_body_ratio times the real body
    if lower_shadow / real_body < shadow_to_body_ratio:
        return NONE

    # Check if the upper shadow is negligible
    if upper_shadow / total_range > upper_shadow_ratio:
        return NONE

    # Check if the last 10 days MA is greater than the current price
    if df.iloc[-10:]['mid_c'].mean() < close_price:
        return NONE

    # Check if the last 200 days MA is less than the current price
    if df.iloc[-200:]['mid_c'].mean() > close_price:
        return NONE

    return BUY

In [59]:
def apply_hammer_check(df):
    """
    Apply the hammer check function to the DataFrame.

    Parameters:
        df: DataFrame containing the candle data (at least 200 rows).

    Returns:
        A new DataFrame with a 'Signal' column indicating BUY or None.
    """
    signals = []
    for i in range(200, len(df)):  # Start from the 200th row to ensure 200 rows of history
        current_row = df.iloc[i]
        sub_df = df.iloc[i - 200:i]  # Last 200 rows
        signal = check_for_hammer(current_row, sub_df)
        signals.append(signal)

    # Fill the first 200 rows with None, as they lack sufficient historical data
    signals = [None] * 200 + signals
    df['Signal'] = signals
    return df


In [60]:

# Example usage:
df = apply_hammer_check(df)


In [61]:
df[df['Signal'] == BUY].shape

(57, 8)

In [62]:
cp = CandlePlot(df.iloc[194:214])
cp.show_plot()